In [ ]:
import torch

datalab: bool = False # wheter the notebook is running on datalab gcp (True) or locally on mac with mps (False)
resume_from_checkpoint: bool = False # resume from last cp. If True; set checkpoint path to an existing checkpoint

do_lora:bool = False # whether to do lora fine tuning or juste last layer fine tuning
device: torch.device = torch.device("cuda") if datalab else torch.device("mps")
model_name: str = "meta-llama/Llama-3.2-1B-Instruct"
torch_dtype: torch.dtype = torch.bfloat16
max_new_tokens:int  = 50
output_dir: str = "../bucket/results_04_04_25" if datalab else "./results"
checkpoint_path: str = "../bucket/results/checkpoint-" if datalab else "/Users/mgg/dev/projets/fine-tuning/cp/checkpoint-11750"
path_dataset: str = "./data/boosted_data.json"

In [ ]:
# source : https://colab.research.google.com/drive/1DqKNPOzyMUXmJiJFvJITOahVDxCrA-wA#scrollTo=9Ixtdtpgyv_a

from transformers import AutoModelForCausalLM, AutoTokenizer

# loads generative model and tokenizer
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer = AutoTokenizer.from_pretrained(model_name, torch_dtype=torch_dtype)
tokenizer.pad_token = tokenizer.eos_token # add a padding token, otherwise it raises an error

In [ ]:
from transformers.pipelines import pipeline
# loads pipeline to keep a view on not fine tuned model

raw_model_pipeline = pipeline("text-generation", model=model_name, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

## 1 - Loads the training dataset in a hugging face Dataset

In [ ]:
import json
from datasets import Dataset
import random

with open(path_dataset, "rt") as f:
    boosted_data = json.load(f)

data_prop = .05
boosted_data = boosted_data[:int(data_prop*len(boosted_data))]
print(f"Length of dataset : {len(boosted_data)}")

tokenized_conversations = tokenizer.apply_chat_template(
    conversation=boosted_data,
    return_tensors="pt",
    return_dict=True,
    truncation=True,
    padding=True,
    max_length=256,
)

tokenized_conversations["labels"] = tokenized_conversations["input_ids"]

conv_idx_for_test: int = random.randint(0, len(boosted_data)-1) # take one conversation for test
test_conv = boosted_data[conv_idx_for_test]

train_dataset = Dataset.from_dict(tokenized_conversations)

print(f"Example of conversation : {test_conv}")

In [ ]:
# view on dataset

train_dataset, train_dataset[91]

## 2 - Training

In [ ]:
from training_tools import print_trainable_parameters, CustomCallback

def last_layers_fine_tuning(model):

    # trick to speed up training : freeze all layers except the last one
    for name, param in model.named_parameters():
        # print(f"{name}   Modelsize: {param.numel()/1000**2:.1f}M parameters")
        if "15" not in name:
            param.requires_grad = False
    return model

In [ ]:
from peft import LoraConfig, get_peft_model

def get_peft_config():
    return LoraConfig(
        r=10,
        lora_alpha=32,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
        lora_dropout=0.1,
        bias="lora_only",
        modules_to_save=["decode_head"]
    )

def lora_fine_tuning(model):
    config = get_peft_config()
    lora_model = get_peft_model(model, config)
    return lora_model

In [ ]:
from trl import SFTConfig, SFTTrainer

to_train_model = lora_fine_tuning(model) if do_lora else last_layers_fine_tuning(model)
print_trainable_parameters(to_train_model)



# Initialize trainer
training_args = SFTConfig(
    output_dir="./results",
    # max_steps=100,
    num_train_epochs=4,
    per_device_train_batch_size=4,
    learning_rate=9.2e-4 if do_lora else 3e-5,
    logging_steps=20,
    # save_steps=100,
    # eval_strategy="steps",
    # eval_steps=50,
)

trainer = SFTTrainer(
    model=to_train_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    peft_config=get_peft_config() if do_lora else None,
)

ft_model_pipeline = pipeline("text-generation", model=trainer.model, tokenizer=tokenizer, max_new_tokens=max_new_tokens)

trainer.add_callback(CustomCallback(raw_model_pipeline=raw_model_pipeline, ft_model_pipeline=ft_model_pipeline, test_conv=test_conv))

In [ ]:
trainer.train()

## Test the model

In [ ]:
model.eval() # eval mode : stops useless gradient computations

In [ ]:
def q_a(question):
    return ft_model_pipeline([{
        "role": "user",
        "content": question
    }])[0]["generated_text"][1]["content"]

In [ ]:
for i in range(10):
    print(q_a("What is Juropa ?")) 
    print("--------\n")

In [ ]:
q_a("What is Juropa in the field of cooking ?") # small check for overfitting